In [1]:
import pandas as pd
import numpy as np

C:\Users\huzef\AppData\Roaming\Python\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\huzef\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\huzef\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies = movies[['movie_id','title','overview','keywords','genres','cast','crew']]

In [ ]:
movies.dropna(inplace=True)

In [ ]:
import ast
def converter(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'].replace(" ",""))
    return L

def converter3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 4:
            L.append(i['name'].replace(" ",""))
            counter = counter + 1
        else:
            break
    return L
def get_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'].replace(" ",""))
            break
    return L
def list_converter(obj):
    return [obj]

movies['overview'] = movies['overview'].apply(list_converter)
movies['keywords'] = movies['keywords'].apply(converter)
movies['genres'] = movies['genres'].apply(converter)
movies['cast'] = movies['cast'].apply(converter3)
movies['crew'] = movies['crew'].apply(get_director)

In [ ]:
movies['tags'] = (movies['overview'] + movies['keywords'] + movies['genres'] + movies['cast'] + movies['crew']).apply(lambda x: " ".join(x)).apply(lambda x:x.lower())

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        movie_id = i[0]
        print(movie_id)
        print(new_df.iloc[i[0]].title)

In [ ]:
import pickle
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
recommend('Avatar')